In [84]:
import numpy as np
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, ReLU
from tensorflow.keras.optimizers import Adam
from ultralytics import YOLO  # Ensure you have installed the YOLOv8 library
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [ ]:
model = YOLO("yolo11n.pt")
model.train(data="coco8.yaml",epochs = 100, imgsz = 400)

In [56]:
results = model('./datasets/Alex/Alex-Image02.png')


image 1/1 /Users/jlamkin/Code/DATA/data403-project3/datasets/Alex/Alex-Image02.png: 416x288 1 person, 51.5ms
Speed: 1.1ms preprocess, 51.5ms inference, 3.5ms postprocess per image at shape (1, 3, 416, 288)


In [57]:
def extract_class_count(model, image_path):
    """
    Extract a dictionary of class counts for detections in an image.
    Args:
        image_path (str): Path to the image.
    Returns:
        dict: A dictionary with class IDs as keys and their counts as values.
    """
    results = model(image_path)
    detections = results[0].boxes.data.cpu().numpy()  # YOLO detections

    # Count occurrences of each class ID
    class_counts = {}
    for box in detections:
        class_id = int(box[5])  # Class ID
        class_counts[class_id] = class_counts.get(class_id, 0) + 1

    return class_counts


In [58]:
def get_unique_classes(model, image_paths):
    """
    Determine the set of unique class IDs across the dataset.
    Args:
        image_paths (list): List of image paths.
    Returns:
        list: Sorted list of unique class IDs.
    """
    unique_classes = set()
    for path in image_paths:
        class_counts = extract_class_count(model, path)
        unique_classes.update(class_counts.keys())
    return sorted(unique_classes)  # Ensure consistent ordering


In [59]:
def class_count_to_vector(class_counts, unique_classes):
    """
    Convert class counts to a fixed-length vector based on unique classes.
    Args:
        class_counts (dict): Dictionary of class counts for an image.
        unique_classes (list): List of unique class IDs (fixed order).
    Returns:
        list: Fixed-length vector representation.
    """
    vector = [0] * len(unique_classes)
    for class_id, count in class_counts.items():
        if class_id in unique_classes:
            index = unique_classes.index(class_id)
            vector[index] = count
    return vector


In [60]:
def get_paths(directory):
    file_paths = []
    for path in os.listdir(directory):
        image_path = os.path.join(directory, path)
        if os.path.isfile(image_path):
                file_paths.append(image_path)
    return file_paths

In [61]:
paths_alex, paths_kelly = get_paths('./datasets/Alex/'), get_paths('./datasets/Kelly')
paths_combined = paths_alex + paths_kelly
unique_classes = get_unique_classes(model, paths_combined)
num_unique = len(unique_classes) #There are 56 total unique classes when run on paths_combined
# for path in paths_alex:
#     class_counts = extract_class_count(path)



image 1/1 /Users/jlamkin/Code/DATA/data403-project3/datasets/Alex/Alex-Image119.png: 320x416 (no detections), 99.1ms
Speed: 1.0ms preprocess, 99.1ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 416)

image 1/1 /Users/jlamkin/Code/DATA/data403-project3/datasets/Alex/Alex-Image131.png: 320x416 1 motorcycle, 3 potted plants, 48.1ms
Speed: 1.3ms preprocess, 48.1ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 416)

image 1/1 /Users/jlamkin/Code/DATA/data403-project3/datasets/Alex/Alex-Image125.png: 320x416 1 train, 45.1ms
Speed: 1.2ms preprocess, 45.1ms inference, 0.7ms postprocess per image at shape (1, 3, 320, 416)

image 1/1 /Users/jlamkin/Code/DATA/data403-project3/datasets/Alex/Alex-Image247.png: 288x416 (no detections), 44.1ms
Speed: 0.9ms preprocess, 44.1ms inference, 0.3ms postprocess per image at shape (1, 3, 288, 416)

image 1/1 /Users/jlamkin/Code/DATA/data403-project3/datasets/Alex/Alex-Image27.png: 320x416 (no detections), 56.4ms
Speed: 0.8ms prep

In [62]:
def class_count_to_vector(class_counts, unique_classes):
    """
    Convert class counts to a fixed-length vector based on unique classes.
    Args:
        class_counts (dict): Dictionary of class counts for an image.
        unique_classes (list): List of unique class IDs (fixed order).
    Returns:
        list: Fixed-length vector representation.
    """
    vector = [0] * len(unique_classes)
    for class_id, count in class_counts.items():
        if class_id in unique_classes:
            index = unique_classes.index(class_id)
            vector[index] = count
    return vector

In [63]:
class_count_to_vector(extract_class_count(model,'datasets/Alex/Alex-Image01.png'), unique_classes)



image 1/1 /Users/jlamkin/Code/DATA/data403-project3/datasets/Alex/Alex-Image01.png: 416x416 1 person, 1 bird, 52.8ms
Speed: 1.2ms preprocess, 52.8ms inference, 0.5ms postprocess per image at shape (1, 3, 416, 416)


[1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [64]:
print(len(paths_alex))
print(len(paths_kelly))
#len(paths_combined)
y = np.concatenate((np.zeros(256),np.ones(229)),axis=0) #ALEX = 0, KELLY = 1
X = []
for path in paths_combined:
    class_counts = extract_class_count(model,path)
    vector = class_count_to_vector(class_counts, unique_classes)
    X.append(vector)
X = np.array(X)
print(X)


256
229

image 1/1 /Users/jlamkin/Code/DATA/data403-project3/datasets/Alex/Alex-Image119.png: 320x416 (no detections), 39.8ms
Speed: 1.1ms preprocess, 39.8ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 416)

image 1/1 /Users/jlamkin/Code/DATA/data403-project3/datasets/Alex/Alex-Image131.png: 320x416 1 motorcycle, 3 potted plants, 42.0ms
Speed: 0.9ms preprocess, 42.0ms inference, 0.7ms postprocess per image at shape (1, 3, 320, 416)

image 1/1 /Users/jlamkin/Code/DATA/data403-project3/datasets/Alex/Alex-Image125.png: 320x416 1 train, 44.3ms
Speed: 0.8ms preprocess, 44.3ms inference, 0.7ms postprocess per image at shape (1, 3, 320, 416)

image 1/1 /Users/jlamkin/Code/DATA/data403-project3/datasets/Alex/Alex-Image247.png: 288x416 (no detections), 35.9ms
Speed: 1.1ms preprocess, 35.9ms inference, 0.3ms postprocess per image at shape (1, 3, 288, 416)

image 1/1 /Users/jlamkin/Code/DATA/data403-project3/datasets/Alex/Alex-Image27.png: 320x416 (no detections), 41.3ms
Speed: 0.

In [ ]:
input_size = len(unique_classes)  # Length of feature vector
seq_model = Sequential([
    Dense(64, input_dim=input_size),
    ReLU(),
    Dense(2, activation='softmax')  # 2 classes: Alex and Kelly
])

# Compile the model
seq_model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
seq_model.fit(X, y, epochs=10, batch_size=2, verbose=1)

Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


243/243 ━━━━━━━━━━━━━━━━━━━━ 1s 792us/step - accuracy: 0.5228 - loss: 0.7056
Epoch 2/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 666us/step - accuracy: 0.6426 - loss: 0.6302
Epoch 3/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 766us/step - accuracy: 0.7076 - loss: 0.5977
Epoch 4/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 623us/step - accuracy: 0.7531 - loss: 0.5580
Epoch 5/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 817us/step - accuracy: 0.6945 - loss: 0.5434
Epoch 6/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 880us/step - accuracy: 0.7423 - loss: 0.5299
Epoch 7/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 699us/step - accuracy: 0.7550 - loss: 0.4880
Epoch 8/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 803us/step - accuracy: 0.7366 - loss: 0.4981
Epoch 9/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 689us/step - accuracy: 0.7306 - loss: 0.4954
Epoch 10/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 616us/step - accuracy: 0.6792 - loss: 0.5251


In [82]:
def make_prediction(prediction):
    if prediction[0] >=.5:
        return 0
    return 1

In [76]:

test_class_counts = extract_class_count(model,'datasets/Alex/Alex-Image03.png')
test_vector = np.array(class_count_to_vector(test_class_counts, unique_classes))

# Ensure the shape includes a batch dimension
test_vector = np.expand_dims(test_vector, axis=0)  # Add batch dimension

# Now pass it to the model
prediction = seq_model.predict(test_vector)
make_prediction(prediction)


image 1/1 /Users/jlamkin/Code/DATA/data403-project3/datasets/Alex/Alex-Image03.png: 320x416 (no detections), 84.0ms
Speed: 18.2ms preprocess, 84.0ms inference, 1.6ms postprocess per image at shape (1, 3, 320, 416)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


0

In [78]:
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(
    X,
    y,
    np.arange(len(X)),  # Use indices to keep track
    test_size=0.20,  # 33% test data
    stratify=y,      # Ensure stratified split
    random_state=42
)

In [85]:
seq_model.fit(X_train, y_train, epochs=10, batch_size=2)
predictions = seq_model.predict(X_test)
results = []
for p in predictions:
    print(p)
    results.append(make_prediction(p))
accuracy = accuracy_score(y_test, results)
print(accuracy)


Epoch 1/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 541us/step - accuracy: 0.7463 - loss: 0.4444
Epoch 2/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 500us/step - accuracy: 0.6796 - loss: 0.4796
Epoch 3/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 499us/step - accuracy: 0.7556 - loss: 0.4473
Epoch 4/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 641us/step - accuracy: 0.7858 - loss: 0.4119
Epoch 5/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 581us/step - accuracy: 0.7523 - loss: 0.4736
Epoch 6/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 585us/step - accuracy: 0.7701 - loss: 0.4270
Epoch 7/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 624us/step - accuracy: 0.7394 - loss: 0.4481
Epoch 8/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 618us/step - accuracy: 0.7236 - loss: 0.4737
Epoch 9/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 653us/step - accuracy: 0.7138 - loss: 0.4493
Epoch 10/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 576us/step - accuracy: 0.7523 - loss: 0.4266
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 894us/step
[  0.0038243     0.99618]
[    0.29491     0.70509]
[    0.60604     0.39396]
[ 